<a href="https://colab.research.google.com/github/jhuang12/Tensorflow-for-personality-items-classification/blob/master/191216TF_IPIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#THIS NOTEBOOK WAS CREATED TO USE TF NLP FOR IPIP ITEMS

In [0]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf

In [2]:
print(tf.__version__)

2.0.0


In [18]:
#!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 52kB/s 
     |████████████████████████████████| 3.8MB 48.4MB/s 
     |████████████████████████████████| 450kB 44.4MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalle

DATA LOADING

In [3]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!pip install pydrive

In [0]:
# dpath = os.path.join(r"C:/Users/amead/Google Drive/active/machine learning/big5/big_five_items.csv")
dpath = os.path.join("/content/drive/My Drive/big_five_items.csv")

In [0]:
data = pd.read_csv(dpath,low_memory=False, sep ="\t", lineterminator='\n', encoding='utf-8')

In [0]:
#with open(dpath, mode='rb') as f:
#    print(f.read())

b'#ID\tScale Item ID\tScale\tDirection\tStem\tInstrument\r\n1\t1\tNeuroticism\t-\tI seldom feel blue.\tBFAS\r\n2\t2\tAgreeableness\t-\tI am not interested in other people\'s problems.\tBFAS\r\n3\t3\tConscientiousness\t+\tI carry out my plans.\tBFAS\r\n4\t4\tExtraversion\t+\tI make friends easily.\tBFAS\r\n5\t5\tOpenness\t+\tI am quick to understand things.\tBFAS\r\n6\t6\tNeuroticism\t+\tI get angry easily.\tBFAS\r\n7\t7\tAgreeableness\t+\tI respect authority.\tBFAS\r\n8\t8\tConscientiousness\t-\tI leave my belongings around.\tBFAS\r\n9\t9\tExtraversion\t+\tI take charge.\tBFAS\r\n10\t10\tOpenness\t+\tI enjoy the beauty of nature.\tBFAS\r\n11\t11\tNeuroticism\t+\tI am filled with doubts about things.\tBFAS\r\n12\t12\tAgreeableness\t+\tI feel others\' emotions.\tBFAS\r\n13\t13\tConscientiousness\t-\tI waste my time.\tBFAS\r\n14\t14\tExtraversion\t-\tI am hard to get to know.\tBFAS\r\n15\t15\tOpenness\t-\tI have difficulty understanding abstract ideas.\tBFAS\r\n16\t16\tNeuroticism\t-\tI r

In [6]:
data.head()

,#ID,Scale Item ID,Scale,Direction,Stem,Instrument
0,1,1,Neuroticism,-,I seldom feel blue.,BFAS
1,2,2,Agreeableness,-,I am not interested in other people's problems.,BFAS
2,3,3,Conscientiousness,+,I carry out my plans.,BFAS
3,4,4,Extraversion,+,I make friends easily.,BFAS
4,5,5,Openness,+,I am quick to understand things.,BFAS


DATA PROCESSING


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 6 columns):
#ID              655 non-null int64
Scale Item ID    655 non-null object
Scale            655 non-null object
Direction        655 non-null object
Stem             655 non-null object
Instrument       655 non-null object
dtypes: int64(1), object(5)
memory usage: 30.8+ KB


In [8]:
data.shape

(655, 6)

In [0]:
#need to add random.sample() into the data so that the data are random
#the beter solution would be to do stratefied sampleing with five dimensions

In [20]:
#stem as one string
sentences = pd.Series(data['Stem'])
sentences

0                                    I seldom feel blue.
1        I am not interested in other people's problems.
2                                  I carry out my plans.
3                                 I make friends easily.
4                       I am quick to understand things.
                             ...                        
650                       I'm pretty stable emotionally.
651                                      I laugh easily.
652    I believe that the "new morality" of a permiss...
653    I would rather be known as "merciful" than as ...
654            I think twice before I answer a question.
Name: Stem, Length: 655, dtype: object

In [10]:
#scale as categorical data
data['labels'] = data['Scale'].astype('category').cat.codes
data.head()

,#ID,Scale Item ID,Scale,Direction,Stem,Instrument,labels
0,1,1,Neuroticism,-,I seldom feel blue.,BFAS,3
1,2,2,Agreeableness,-,I am not interested in other people's problems.,BFAS,0
2,3,3,Conscientiousness,+,I carry out my plans.,BFAS,1
3,4,4,Extraversion,+,I make friends easily.,BFAS,2
4,5,5,Openness,+,I am quick to understand things.,BFAS,4


In [11]:
labels = np.array(pd.Series(data['labels']))
labels

array([3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0,
       1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2,
       4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3,
       0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1,
       2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4,
       2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0, 1, 3, 4, 2, 0,
       1, 3, 4, 2, 0, 1, 3, 4, 4, 0, 1, 4, 4, 2, 2, 0, 2, 0, 3, 4, 4, 2,
       3, 2, 2, 2, 3, 2, 4, 1, 3, 1, 1, 0, 0, 1, 4, 1, 3, 0, 0, 0, 4, 4,
       4, 0, 2, 3, 4, 0, 1, 0, 1, 0, 3, 1, 2, 3, 3, 1, 2, 2, 3, 0, 2, 1,
       3, 3, 4, 0, 1, 1, 2, 0, 4, 0, 2, 1, 3, 0, 4, 1, 4, 1, 3, 1, 3, 1,
       3, 1, 1, 1, 2, 4, 4, 3, 3, 1, 4, 0, 4, 2, 2, 3, 4, 3, 3, 0, 0, 2,
       3, 1, 3, 1, 1, 1, 3, 1, 2, 0, 4, 0, 2, 1, 4, 1, 2, 0, 2, 3, 4, 2,
       3, 0, 2, 0, 4, 3, 4, 4, 4, 0, 2, 4, 4, 0, 1, 1, 4, 0, 3, 0, 1, 2,
       3, 1, 2, 3, 0, 2, 0, 0, 4, 1, 3, 3, 3, 3, 4,

DATA CLEANING (STEMMING)

TOKENIZATION AND EMBEDDING

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token = "<OOV>", num_words = 10000)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding = 'pre')
print(padded[0])
print(padded.shape)

{'<OOV>': 1, 'i': 2, 'to': 3, 'a': 4, 'of': 5, 'am': 6, 'and': 7, 'my': 8, 'the': 9, 'have': 10, 'in': 11, 'people': 12, 'things': 13, 'be': 14, 'others': 15, 'feel': 16, 'get': 17, 'is': 18, 'with': 19, 'not': 20, "i'm": 21, 'or': 22, 'easily': 23, 'for': 24, 'me': 25, 'myself': 26, 'that': 27, 'like': 28, 'do': 29, 'when': 30, 'often': 31, 'on': 32, 'sometimes': 33, 'can': 34, 'as': 35, 'about': 36, 'at': 37, 'lot': 38, 'it': 39, 'work': 40, "don't": 41, 'them': 42, 'make': 43, 'take': 44, 'rarely': 45, 'believe': 46, 'than': 47, 'person': 48, 'think': 49, 'seldom': 50, 'time': 51, 'know': 52, 'up': 53, 'other': 54, 'keep': 55, 'feelings': 56, 'if': 57, 'ideas': 58, 'find': 59, 'very': 60, 'most': 61, 'try': 62, 'enjoy': 63, 'what': 64, 'by': 65, 'good': 66, 'done': 67, 'an': 68, 'always': 69, 'are': 70, 'around': 71, 'love': 72, 'way': 73, 'into': 74, 'would': 75, 'just': 76, 'problems': 77, "others'": 78, 'emotions': 79, 'being': 80, 'difficult': 81, 'little': 82, 'avoid': 83, 'min

TRAINING

In [0]:
#split training and validation sample
training_data, training_labels = np.asarray(padded[ :524]), labels[ :524]
test_data, test_labels = np.asarray(padded[525: ]), labels[525: ]

MODEL PERFORMANCE

MODEL 

In [53]:
training_data

array([[  0,   0,   0, ...,  50,  16, 103],
       [  0,   0,   0, ...,  54, 126,  77],
       [  0,   0,   0, ..., 105,   8, 165],
       ...,
       [  0,   0,   0, ..., 783,  19, 289],
       [  0,   0,   0, ...,   7, 784, 394],
       [  0,   0,   0, ...,  42,  52,  39]], dtype=int32)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 64),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu')
])

In [74]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 64)          64000     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         8320      
_________________________________________________________________
global_average_pooling1d_5 ( (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 325       
Total params: 80,901
Trainable params: 80,901
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
#NUM_EPOCHS = 10
history = model.fit(training_data, training_labels,
                    #test_data, test_labels,
                    epochs=100)

Train on 524 samples
Epoch 1/100
524/524 [==============================] - 1s 1ms/sample - loss: 2.2167 - accuracy: 0.1927
Epoch 2/100
524/524 [==============================] - 0s 154us/sample - loss: 1.6242 - accuracy: 0.2004
Epoch 3/100
524/524 [==============================] - 0s 167us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 4/100
524/524 [==============================] - 0s 169us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 5/100
524/524 [==============================] - 0s 166us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 6/100
524/524 [==============================] - 0s 156us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 7/100
524/524 [==============================] - 0s 158us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 8/100
524/524 [==============================] - 0s 162us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 9/100
524/524 [==============================] - 0s 193us/sample - loss: 1.6094 - accuracy: 0.2004
Epoch 10/100
524/524 [==============

In [0]:
checkponnt

In [0]:
model.save()

In [0]:
model.predict()